# DOC


## DOCUMENTATION
___
### **LLM**

| **Model**                    | **Context Window** |**Limits**           |
|------------------------------|--------------------|---------------------|
| **Qwen 2.5 35B Code**        | Up to 128K tokens  |Fine Tuned for coding|
| **Cohere Command R+**        | Up to 128K tokens  | 1000 calls per month|
| **Mistral-7B-Instruct-v0.3** | 32,768 tokens      | Hallucinations      |

#### Using `Mistral-7B-Instruct-v0.3` because 
- reliable outputs
- 1000 calls per day
- works in huggingface(same as text-to-image)


### **Text-to-Image**


| **Category**               | **Stable Diffusion 3.5 Large Turbo**                                                                 | **FLUX.1-Dev FP16**                                                                       |
|----------------------------|-----------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| **API Endpoint**           | `stabilityai/stable-diffusion-3-5-large-turbo`                                                     | `black-forest-labs/FLUX.1-Dev-FP16`                                                     |
| **Key API Parameters**     | `guidance_scale=3`, `num_inference_steps=4`, `negative_prompt`                                     | `num_inference_steps=8`, `control_image` (Canny/Depth), `true_cfg=4.0`                  |
| **API Latency**            | 3.8–5.2 sec/image (cold start: 12–18 sec)                                                          | 6.1–8.9 sec/image (cold start: 15–22 sec)                                                |
| **Cost Efficiency**        | $0.0021/image (PRO tier)                                                                           | $0.0033/image (PRO tier)                                                                 |
| **Free Tier Limits**       | 500 requests/hour                                                                                  | 300 requests/hour                                                                        |
| **Max Resolution**         | 1024x1024 via single API call                                                                      | 2048x2048 (requires `high_res_fix=true` parameter)                                       |
| **Advanced Features**      | - 4-step inference <br> - Text-to-image only                                                       | - Unified ControlNet (Canny/Depth) <br> - Image-to-image <br> - Inpainting/Outpainting   |
| **NSFW Filtering**         | Enabled by default (`safety_checker=strict`)                                                       | Optional (`safety_checker=relaxed`)                                                      |
| **Rate Limits (PRO)**      | 5K requests/hour                                                                                   | 3K requests/hour                                                                         |
| **Use Case Focus**         | Rapid batch generation (social media, prototyping)                                                 | High-detail workflows (product design, architectural viz)                                |

#### using `SD-3.5-LT` because 
- latency is low
- More requests
- we want stialized images which is better on SD 



___
___
___

# Groqq


### Get **Summary**, **Characters** AND **Places** 
![Local Image](./media/Sudo_summary.png "Smmary, characters and Places")

In [1]:
#IMPORTS
from reader import ebook,read_list,read_json
from typing import Optional,Dict,List
import requests
from dotenv import load_dotenv
import json
import os
from pydantic import BaseModel, ValidationError
from dataclasses import dataclass
from groq import Groq

In [2]:
#model init 

load_dotenv()
API = os.getenv("HF_API")
GROQ= os.getenv("GROQ_API")


groq_client=Groq(api_key=GROQ,)

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}

url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"

In [3]:
sum_role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and Ready to parse. nothing before or after the json file)
Input:  
1.Current Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their physical/visual descriptions till now (This chapter).
3.Places list: list of places and their visual description till now (This chapter).
4.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize and explain the chapter in detail, integrating context and key developments from previous chapters and create a self containing summary and explaination. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character's physical/visual descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary and explination of the current chapter in context of previous chapters. Use previus chapter summary as context",
  "characters": {
      "Character Name": "Updated or new physical/visual description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new visual description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}
'''

### **get_sum()** Function
![Local Image](./media/Get_sum_logic.png "Smmary, characters and Places")

In [22]:

class SummarySchema(BaseModel):
    summary: str
    characters: Dict[str,str]
    places: Dict[str,str]

def sum_msg(text: str, role:str, context: str, characters: dict = {}, places: dict = {}) -> list:
    message =[
        {
            "role": "system",
            "content": f"{role}, make sure the out follow the given json schema{json.dumps(SummarySchema.model_json_schema(), indent=2)}",
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "Current_Chapter": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message
    
def groq_sum(msg:str,validator:BaseModel):
    
    try:
        completion = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=msg,
            temperature=1,
            max_completion_tokens=8000,
            top_p=1,
            stream=False,
            stop=None,
            response_format={"type": "json_object"},
        )
        
    except Exception as e:
        return (f"Request error : {e}")
    
    try:
        validator.model_validate_json(completion.choices[0].message.content)
        return completion.choices[0].message.content
    except Exception as e:
        return f"Validation error : {e}"

In [20]:
#text output_dict
book=ebook("./books/stranger.pdf")
title=book.get_metadata()['title']
_,text=zip(*book.get_chapters())
text=text[:5]

output_dict={
    0:
        {
            "summary":"There is no previous context",
            "characters":{},
            "places": {}
        }
            }

In [24]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt")  # Download tokenizer model if needed

for i in text:
    tokens = word_tokenize(i)  # Better handling of punctuation
    print(len(tokens))  # 8 (counts punctuation as separate tokens)



490
6491
2685
4337
2365


[nltk_data] Downloading package punkt to /home/prince/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
for idx,i in enumerate(text):
 
    if len(i)<1200:
        output_dict[idx+1]=output_dict[idx]
    else:
        context=output_dict[idx]["summary"]
        characters=output_dict[idx]["characters"]
        places=output_dict[idx]["places"]
        
        mes=sum_msg(
            text=i,
            role=sum_role,
            context=context,
            characters=characters,
            places=places
            )
        
        summary_characters=groq_sum(mes,validator=SummarySchema)
        
        output_json=read_json(summary_characters)
        output_dict[idx+1]=output_json
        
    print(f"chapter: {idx} done")
print(output_dict)

chapter: 0 done
Error: Expecting value: line 1 column 1 (char 0)



Request error : Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01jb0ynhzvfppr48n0wg2cxn9b` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 8613, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
chapter: 1 done


TypeError: 'NoneType' object is not subscriptable

In [17]:
#print out_dict
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Narrator: The narrator is a grown-up who lives in France, feels hungry and cold, and is reminiscing about his childhood. He is a pilot who has traveled the world and has had many interactions with grown-ups.
Leon Werth: Leon Werth is the friend to whom the narrator dedicates his book, and is remembered by the narrator as a little boy.
The little voice: A mysterious voice that wakes the narrator up in the Sahara desert and asks him to draw a sheep.

Places:
France: The country where the narrator lives, and where he feels hungry and cold.
The virgin forest: A jungle where boa constrictors live, which the narrator read about in a book called Stories of Life when he was six years old.
The Sahara desert: A desert where the narrator had a breakdown six years ago, and where he was stranded with no one to talk to.

Summary:
The narrator, who lives in France and feels hungry and cold, dedicates his book to his friend Leon Werth when he was a little boy. He reminisces ab

### Get **Scenes**
![Local Image](./media/Sudo_summary.png )

In [18]:
class SceneSchema(BaseModel):
    scenes: Dict[str,str]
print(SceneSchema.model_json_schema())

{'properties': {'scenes': {'additionalProperties': {'type': 'string'}, 'title': 'Scenes', 'type': 'object'}}, 'required': ['scenes'], 'title': 'SceneSchema', 'type': 'object'}


In [19]:
scene_role=''' 
Analyze the input text and:  
1. Detect Scene Changes: Identify shifts in location, time, or significant events.  
2. Generate Scene Prompts: For each scene, create a text-to-image prompt capturing:  
   - Subject: Characters, objects, appearances.  
   - Setting: Location, time, weather, lighting.  
   - Action/Emotion: Key actions and emotional tone.  
   - Symbolism/Subtext: Hidden themes or foreshadowing.  
   - Style: Artistic medium, color palette, composition.  
3. Output JSON: Return a JSON list   

Key Features 
1. Scene Detection: Splits input into scenes based on time/location shifts (e.g., "next morning," "that night").  
2. Precision: Captures character details, environmental cues, and subtext (e.g., owls = magical intrusion).  
3. Style Flexibility: Specifies artistic styles (realism, fantasy) and references (Hopper, Crewdson).  
4. JSON Ready: Outputs a parseable list for automated workflows.  

Output Format:  
{ 
  "scene":"[Detailed text-to-image prompt]",  
}  

Example Input:  
"The Dursleys lay in bed, Vernon awake and anxious while Petunia slept. The next morning, he walked to work, spotting cloaked figures whispering on the street. That night, owls flooded the sky, dropping letters on rooftops."

Example Output:  
{
    "scene1":"Highly detailed digital painting of a dim suburban bedroom at night. A middle-aged man (Vernon Dursley) in stiff pajamas lies rigidly awake, hands clasped over his chest, staring at the ceiling with a furrowed brow. Beside him, a plump woman (Petunia Dursley) sleeps under a floral quilt, her face pinched. Faint orange streetlamp glow filters through half-closed curtains, casting shadows on dated wallpaper. Rain streaks the window, revealing distant storm clouds. Mood: Tense and uneasy. Style: Realistic with muted tones (slate grays, mustard yellows), chiaroscuro lighting, Edward Hopper-inspired composition.",
    "scene2": "Photorealistic morning street scene. Vernon Dursley, in a crisp suit, strides down a damp suburban sidewalk, glancing nervously at cloaked figures whispering in the background. Overcast sky, puddles reflecting muted light. Mood: Suspicious and mundane. Style: Cool gray and green tones, wide-angle shot, Gregory Crewdson-inspired cinematic realism." ,
    "scene3": "Fantasy night scene. Hundreds of owls darken the sky, swooping over rooftops and dropping envelopes. Moonlight illuminates swirling feathers and letters raining onto a suburban neighborhood. Mood: Surreal and ominous. Style: Dark fantasy digital painting, deep blues and silvers, dynamic motion blur, inspired by John Howe.",
}  
'''

In [20]:
def scene_msg(text: str) -> list:
    message = [
        {
            "role": "system",
            "content":f''' {scene_role} make sure the output follow the given json schema  {json.dumps(SceneSchema.model_json_schema(), indent=2)}''',
        },
        
        {
            "role": "user",
            "content": text,  # This should be your input text that describes the scenes
        },]
        
    return message

In [21]:
scene_output_list=[]
print("--Genrating Scenes per chapter--")
for idx,i in enumerate(text):
    inputs=i.replace("\n"," ")
    scene_message = scene_msg(inputs)
    scene_output=groq_sum(msg=scene_message,validator=SceneSchema)
    if scene_output is None:
        print("None")
        continue
    print(scene_output)
    scene_json_output=read_json(scene_output)
    scene_output_list.append(scene_json_output)
    print(f"chapter: {idx+1} Done")

--Genrating Scenes per chapter--
{
   "scenes": {
      "scene1": "A watercolor illustration of a child's bedroom at night, with a six-year-old boy sitting at a desk, drawing with a colored pencil. A book lies open in front of him, showing a picture of a boa constrictor swallowing a wild beast. The room is dimly lit, with a single bedside lamp casting a warm glow. Mood: Curious and imaginative. Style: Whimsical, with loose brushstrokes and muted colors, inspired by Beatrix Potter.",
      "scene2": "A realistic depiction of a boa constrictor digesting an elephant, drawn in a child's scribble style. The boa constrictor is shown with a large lump in its body, and the elephant's legs and tail are visible inside. The background is a warm, sunny yellow, representing the jungle. Mood: Playful and humorous. Style: Naive, with bold lines and bright colors, inspired by children's drawings.",
      "scene3": "A portrait of the narrator as a grown-up, standing in front of a map of the world, wear

In [46]:
outputs=[]
# scene_output_list[0]['scenes']
for i in scene_output_list:
    # print(i["scenes"].values())
    outputs.append(list(i["scenes"].values()))
outputs

[["A watercolor illustration of a child's bedroom at night, with a six-year-old boy sitting at a desk, drawing with a colored pencil. A book lies open in front of him, showing a picture of a boa constrictor swallowing a wild beast. The room is dimly lit, with a single bedside lamp casting a warm glow. Mood: Curious and imaginative. Style: Whimsical, with loose brushstrokes and muted colors, inspired by Beatrix Potter.",
  "A realistic depiction of a boa constrictor digesting an elephant, drawn in a child's scribble style. The boa constrictor is shown with a large lump in its body, and the elephant's legs and tail are visible inside. The background is a warm, sunny yellow, representing the jungle. Mood: Playful and humorous. Style: Naive, with bold lines and bright colors, inspired by children's drawings.",
  "A portrait of the narrator as a grown-up, standing in front of a map of the world, wearing a pilot's uniform and holding a pair of binoculars. The map shows various locations mark

### Image

### Loop Logic
![Local Image](./media/Image_Loop.png )

In [49]:
from huggingface_hub import InferenceClient
import time


In [50]:
client = InferenceClient(
    # model="strangerzonehf/Qs-Sketch",
    model="strangerzonehf/Flux-Sketch-Sized-LoRA",
    token=API,
)

In [56]:
def get_images(key, text,tag,characters,places):
    image = client.text_to_image(
        f"Text: {text}// context-> characters: {characters}, places:{places}",
        # negative_prompt="hand,feet,text,written,shinny,artificial,unnatural,plastic,words,letters",
        height=480,
        width= 480,
        # guidance_scale=2,
        # num_inference_steps=10
        
    )
    image.save(f"./output/1Sketch_{title}_{key}_{tag}.png")

### Get **Image**
![Local Image](./media/Get_img_logic.png )

In [57]:
for idx,i in enumerate(outputs):
    chars=output_dict[idx+1]["characters"]
    places=output_dict[idx+1]["places"]
    for jdx,j in enumerate(i):
        ##try except for internal server error
        try:
            key=f"C{idx}S{jdx+1}"
            text=j
            tag=""
            get_images(key=key,text=text,tag=tag,characters=chars,places=places)
            print(f"{key} saved. tag : {tag} ")
            print(f"    prompt:{text} \n    characters:{chars}\n    places:{places}")
        except Exception as e:
            print(f"Error while calling the API :{e} \n waiting for 10 seconds.. ")
            time.sleep(10)
            get_images(key=key,text=text,tag=tag,characters=chars,places=places)
            print(f"{key} saved. tag : {tag} ")
            print(f"    prompt:{text}")

C0S1 saved. tag :  
    prompt:Detailed digital painting of a dimly lit cockpit at night. A middle-aged man, the narrator, sits in the pilot's seat, looking out at the stars with a mixture of wonder and unease. The dashboard glows with soft blue lights, and the window reflects the curvature of the Earth. Mood: Contemplative and nostalgic. Style: Realistic with muted tones, inspired by the works of Edward Hopper. 
    characters:{'Narrator': 'The narrator is a grown-up who lives in France, feels hungry and cold, and is reminiscing about his childhood. He is a pilot who has traveled the world and has had many interactions with grown-ups.', 'Leon Werth': 'Leon Werth is the friend to whom the narrator dedicates his book, and is remembered by the narrator as a little boy.', 'The little voice': 'A mysterious voice that wakes the narrator up in the Sahara desert and asks him to draw a sheep.'}
    places:{'France': 'The country where the narrator lives, and where he feels hungry and cold.', '